In [1]:
import lib.aoc.grid2d.grid as grid2d
import lib.aoc.grid2d.vector as vector2d
from collections import defaultdict
from heapq import heappush, heappop

In [2]:
with open('../data/2024/day18.txt') as f:
   data = f.read()

In [3]:
grid_height, grid_width = 71, 71
start, goal = (0, 0), (grid_height - 1, grid_width - 1)
grid = grid2d.parse("")
grid2d.render(start, goal, lambda yx: (grid.update({yx: '.'})), wrap=False)
corrupted_bytes = data.splitlines()

In [4]:
def corrupt_bytes_at_t(t: int):
    return [tuple(map(int, reversed(byte.split(','))))
            for byte in corrupted_bytes[:t]]

def dijkstra():
    cost_from_start = defaultdict(lambda: float('inf'))
    prev_node = defaultdict(None)
    queue = [(0, start)]

    while queue:
        cost, at = heappop(queue)
        if at == goal:
            # Rebuild path
            path, i = [], at

            while True:
                if i is None or i == start: break
                path.append(i)
                i = prev_node[i]

            return cost, set(path + [start])

        for n_at, tile in grid2d.neighbors(grid, at, vector2d.NESW):
            if tile == '#': continue
            new_cost = cost + 1

            if new_cost < cost_from_start[n_at]:
                cost_from_start[n_at] = new_cost
                prev_node[n_at] = at
                heappush(queue, (new_cost, n_at))
    return None, None

In [5]:
# Part 1
grid.update({yx:'#' for yx in corrupt_bytes_at_t(1024)})
part1, best_path = dijkstra()
print("Part 1:", part1)

Part 1: 264


In [6]:
# Part 2
for t in range(part1, len(corrupted_bytes)):
    next_byte = corrupted_bytes[:t][-1:][0]
    next_yx = tuple(map(int, reversed(next_byte.split(','))))

    # If the new byte wasn't on our best path, it obviously didn't interrupt it
    if next_yx not in best_path: continue

    # Once a byte landed on our old best path, calculate a new one
    if t != part1: grid.update({yx:'#' for yx in corrupt_bytes_at_t(t)})
    dist, best_path = dijkstra()

    if dist is None:
        print("Part 2:", next_byte)
        break

Part 2: 41,26
